# Streamlit slider for interactive data selection

Note: Session can be used for several reasons, see more in [Snowflake Session documentation]( https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.25.0/snowpark/api/snowflake.snowpark.Session#snowflake.snowpark.Session).

In [ ]:
# Import python packages
import streamlit as st #allows interaction
import pandas as pd #dataframe support

# get current session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col

Previously, data was loaded from Tasty Bytes dataset (sample data on Snowflake), then demonstrated (in table) for such entries where the company is called 'Freezing Point'.

In [ ]:
df_menu_freezing_point = session.table("tasty_bytes_sample_data.raw_pos.menu")\
.filter(col("truck_brand_name") == 'Freezing Point')
df_menu_freezing_point

Let's find the minimum and maximum of SALE_PRIVE_USD column to know the limits. At first using describe(), but (!) it requires a table (= at least two columns, as shown below).

In [ ]:
df_menu_freezing_point['MENU_TYPE_ID', 'SALE_PRICE_USD'].describe()

In [ ]:
from snowflake.snowpark.functions import min, max
df_menu_freezing_point.agg(min('SALE_PRICE_USD'), max('SALE_PRICE_USD')).show()

Set the (interactive) slider as the minimum value of price to limit/filter results. Sale price ranges from 1 to 20 in the full dataset of Tasty Bytes and ranges from 2 to 7 for company Freezing Point.

In [ ]:
import streamlit as st
st.markdown("## Move the slider to define lower price limit to filter data")
min_price = st.slider('Define min_price', 2, 7, 2)
st.markdown("## Move the slider to define higher price limit to filter data")
max_price = st.slider('Define max_price', 2, 7, 7)

## SQL example
SQL query below runs on the complete database, filtered according to above set minimum and maximum sale price values.

In [ ]:
SELECT truck_brand_name, menu_item_name, sale_price_usd
FROM tasty_bytes_sample_data.raw_pos.menu
WHERE sale_price_usd > {{min_price}} AND sale_price_usd < {{max_price}}

Python code uses above set minimum value (min_price) on the dataset filtered for Freezing Point company. 

In [ ]:
df_menu_freezing_point[df_menu_freezing_point['SALE_PRICE_USD'] > min_price]\
[['TRUCK_BRAND_NAME','MENU_ITEM_NAME','SALE_PRICE_USD']]

or use both limits:

In [ ]:
df_selected = df_menu_freezing_point[(df_menu_freezing_point['SALE_PRICE_USD'] > min_price) \
& (df_menu_freezing_point['SALE_PRICE_USD'] < max_price)]\
[['TRUCK_BRAND_NAME','MENU_ITEM_NAME','SALE_PRICE_USD']]
df_selected

To update limits first move the sliders according to your demands, and then run the cells below (depending on the dynamic values of) the sliders.

Note: that related data tables or visualisation does not change automatically on changing values on the slider(s)!

In [ ]:
df_selected.describe()